In [10]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import re
import csv

In [95]:
#Constants
PATH = '../MOOC/courseware_xml/'
ORIGIN = 'course.xml'

#XML commom attributes
URL = 'url_name'
NAME = 'display_name'
YT_URL = 'youtube_id_1_0'

ELEM_TO_PARSE = ['course','chapter','sequential','vertical','problem','video']
END_RECUR = ['problem','video']

YT_PREFIX = "https://www.youtube.com/watch?v="

In [96]:
tree = ET.parse(PATH + ORIGIN)
root = tree.getroot()


In [65]:
#videos = [[subchapter, url_name, youtube_url], ...]
videos = []

#problems = [[subchapter, url_name], ...]
problems = []

In [72]:

def extract_subchapter(seq):
    title =  seq.attrib[NAME]
    first_word = title.split(' ')[0]
    if re.search("^([0-9]*[.]?)+$",first_word):
        return first_word #return subchapter as a string
    else:
        return 0 #not a subchapter number

In [80]:
def read_node(tag, url, subchapter=0):
#     print("{}: {}".format(child.tag, subchapter))
    curr_tree = ET.parse("{path}/{tag}/{url}.xml".format(path=PATH, tag=tag, url=url))
    curr_root = curr_tree.getroot()

    if curr_root.tag == "sequential":
        subchapter = extract_subchapter(curr_root)
        
    if curr_root.tag == 'video':
        videos.append([subchapter, url,  YT_PREFIX + curr_root.attrib[YT_URL]])
        
    if curr_root.tag == 'problem':
        problems.append([subchapter, url])
        
    if curr_root.tag not in END_RECUR: #Stop recursion on Problems and Videos
        for child in curr_root:
            if child.tag in ELEM_TO_PARSE: #Parse only tags of interest
                read_node(child.tag, child.attrib[URL], subchapter)

In [101]:
def write_csv(videos, problems, suffix=root.attrib[URL]):
    with open("videos_"+suffix+".csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows([["Chapter","VideoID", "Source"]])
        writer.writerows(videos)
    with open("problems_"+suffix+".csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows([["Chapter","ProblemID"]])
        writer.writerows(problems)

In [102]:
videos = []
problems = []
read_node(root.tag, root.attrib[URL])
write_csv(videos,problems)